In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://securit

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product|2015-08-31 00:00:00|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|H

In [ ]:
# Filter Dataframe for total_votes above or equal to 20.
df1 = df.filter(df.total_votes >= 20)
df1.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...|2015-08-31 00:00:00|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|H

In [ ]:
# Filter the DataFrame for helpful_votes ratio above or equal to 50%
df2 = df1.filter((df1.helpful_votes / df1.total_votes) >= 0.5)
df2.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...|2015-08-31 00:00:00|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|H

In [ ]:
# Create paid vine DataFrame
paid_df = df2.filter(df2.vine == "Y")
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   53093730|R29V4UDSS053D8|B00P6ORMR4|     406381545|Aquasana AQ-RO-3....|Home Improvement|          3|           33|         38|   Y|                N|Not for average h...|This filter is ov...|2015-08-30 00:00:00|
|         US|   25984637|R1I2D068WC37PA|B00ZWBURWI|     302288140|Myfox Security Ca...|H

In [ ]:
# Create unpaid Vine Dataframe
unpaid_df = df2.filter(df2.vine == "N")
unpaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42943632|R2VIQ3UX794Q0O|B00YMNSCYM|      74738477|LE LED Under Cabi...|Home Improvement|          4|           21|         21|   N|                Y|Nice Kit, I showe...|Nice Kit, I showe...|2015-08-31 00:00:00|
|         US|   24649241|R1OA24IIHWF54G|B00YI5TNXI|     813068315|Hyperikon BR20-Se...|H

In [ ]:
# Total paid reviews
total_paid_reviews = paid_df.count()
total_paid_reviews

266

In [ ]:
# Total paid 5-star reviews
five_star_paid_reviews = paid_df.filter(paid_df.star_rating == 5).count()
five_star_paid_reviews

125

In [ ]:
# Determine the percetnage of 5 star reviews for paid reviews

five_star_paid_reviews = (five_star_paid_reviews / total_paid_reviews)*100
five_star_paid_reviews

46.99248120300752

In [ ]:
# Total unpaid reviews
total_unpaid_reviews = unpaid_df.count()
total_unpaid_reviews

38829

In [ ]:
# Total unpaid 5-star reviews
five_star_unpaid_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
five_star_unpaid_reviews

18246

In [ ]:
# Determine the percetnage of 5 star reviews for unpaid reviews

five_star_unpaid_reviews = (five_star_unpaid_reviews / total_unpaid_reviews)*100
five_star_unpaid_reviews

0.12101947337637947